In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import imaplib

In [2]:
import email
from email.header import decode_header, make_header

In [3]:
import re

In [4]:
imap = imaplib.IMAP4_SSL("imap.gmail.com")

In [5]:
google_id = "hayangee100@gmail.com"
google_pw = pd.read_csv("./google_passwoed/google_password.csv")
google_pw = list(google_pw)[0]

In [6]:
imap.login(google_id, google_pw)

('OK', [b'hayangee100@gmail.com authenticated (Success)'])

In [7]:
# 사서함 선택, 반환 데이터는 mailbox에 있는 메시지 수
imap.select("INBOX")

('OK', [b'43'])

In [8]:
# 사서함의 모든 메일의 uid 정보 가져오기
# 만약 특정 발신 메일만 선택하고 싶다면 'ALL' 대신에 '(FROM "xxxxx@naver.com")' 입력
status, messages = imap.uid('search', None, 'ALL')
status

'OK'

In [9]:
messages = messages[0].split()

In [10]:
# 0이 가장 마지막 메일(옛날거), -1이 가장 최신 메일
# recent_email = messages[-1]

recent_email = messages[-1]
recent_email

b'318'

In [11]:
# fetch 명령어로 메일 가져오기
res, msg = imap.uid('fetch', recent_email, "(RFC822)")

In [12]:
# 사람이 읽을 수 있는 없는 상태의 이메일
raw = msg[0][1]

In [13]:
# 사람이 읽을 수 있는 형태로 변환
raw_readable = msg[0][1].decode('utf-8')

In [14]:
# raw_readable에서 원하는 부분만 파싱하기 위해 email 모듈을 이용해 변환
email_message = email.message_from_string(raw_readable)

In [15]:
# 보낸사람
fr_header = make_header(decode_header(email_message.get('From')))
print(fr_header)

hayangee100@gmail.com


In [16]:
fr_header = email_message.get('From')
decoded_fr = make_header(decode_header(fr_header))
decoded_fr

In [17]:
# 디코딩된 문자열을 문자열로 변환
fr = str(decoded_fr)
fr

'hayangee100@gmail.com'

In [18]:
match = re.match(r'(.+?)\s*<(.+?)>', fr)

In [19]:
name = ""
email_address = ""
if match:
    name = match.group(1).strip()
    email_address = match.group(2).strip()

    print("Name:", name)
    print("Email Address:", email_address)
else:
    print("패턴이 매치되지 않았습니다.")
    if fr.split("@"):
        fr_split = fr.split("@")
        fr_split = fr_split[0] + "@" + fr_split[1] 
        email_address = fr_split
    else :
        name = fr

패턴이 매치되지 않았습니다.


In [20]:
print(name)
print(email_address)


hayangee100@gmail.com


In [21]:
# 메일 제목
subject = make_header(decode_header(email_message.get('Subject')))
print(subject)

제목: 파이썬으로 Gmail 보내기


In [22]:
body = ""
# 메일 내용
if email_message.is_multipart():
    for part in email_message.walk():
        ctype = part.get_content_type()
        cdispo = str(part.get('Content-Disposition'))
        if ctype == 'text/plain' and 'attachment' not in cdispo:
            body = part.get_payload(decode=True)  # decode
            break
else:
    body = email_message.get_payload(decode=True)

In [23]:
# 내용 부분 디코딩
if body != "":
    body = body.decode('utf-8')
else :
    body = ""
print(body)

내용: 본문 내용


In [24]:
# 미리 만들어둔 빈 데이터 프레임
mail_df = pd.read_csv("./mail.csv")
mail_df

,보낸이,메일주소,제목,내용


In [25]:
# 데이터 프레임으로 만들기
data = {'보낸이': [name], "메일주소" : [email_address], '제목': [subject], '내용': [body]}
df = pd.DataFrame(data)

In [26]:
df

,보낸이,메일주소,제목,내용
0,,hayangee100@gmail.com,제목: 파이썬으로 Gmail 보내기,내용: 본문 내용


In [27]:
mail_df = pd.concat([mail_df, df], ignore_index=True)

In [28]:
mail_df

,보낸이,메일주소,제목,내용
0,,hayangee100@gmail.com,제목: 파이썬으로 Gmail 보내기,내용: 본문 내용


In [29]:
------

SyntaxError: invalid syntax (3549499166.py, line 1)

In [ ]:
# 최종적 저장
mail_df.to_csv("./mail.csv", index = False)

In [ ]:
pd.read_csv("./mail.csv")